In [2]:
import os

import numpy as np
from skimage.feature import peak_local_max
from scipy.stats import multivariate_normal
import progressbar
import matplotlib.pyplot as plt
import imageio

from utoolbox.container import Volume

### Environment presets

In [4]:
file_path = os.path.join(
    *["data", "20170831_SIM", "SI_b1a2DSIM_os", "summedWF", "RAWb1a2DSIM_os_ch0_stack0000_561nm.tif"]
)

In [5]:
kernel_size = 15
radius = (kernel_size-1) // 2

### Load data

In [ ]:
raw = Volume(file_path)
print("shape={}".format(raw.shape))
print("dtype={}".format(raw.dtype))

In [ ]:
print("mean={:.2f}, sd={:.2f}".format(np.mean(raw), np.std(raw)))

### Find peaks

In [ ]:
th = np.mean(raw) + 2*np.std(raw)
coords = peak_local_max(
    raw, exclude_border=radius, min_distance=2*kernel_size, threshold_abs=th
)

In [ ]:
print("{} peaks found".format(len(coords)))

### Isolate the patches

In [ ]:
patches = np.zeros((len(coords), kernel_size, kernel_size, kernel_size), dtype=np.float32)

In [ ]:
index = 0
bar = progressbar.ProgressBar()
for coord in bar(coords):
    x = coord[2]
    y = coord[1]
    z = coord[0]
    
    patches[index, ...] = raw[z-radius:z+radius+1, y-radius:y+radius+1, x-radius:x+radius+1]
    
    index += 1

Save cropped patches.

In [ ]:
patches_folder = os.path.join(os.path.dirname(file_path), "patches")
if not os.path.exists(patches_folder):
    os.makedirs(patches_folder)
    
index = 0
bar = progressbar.ProgressBar()
for patch in bar(patches):
    patches_path = os.path.join(patches_folder, "{}.tif".format(index))
    imageio.volwrite(patches_path, patch)
    index += 1

### Preview

In [ ]:
raw_xy = np.amax(raw, axis=0)

In [ ]:
plt.figure(figsize=(10, 10))
plt.autoscale(enable=True, tight=True)

plt.imshow(raw_xy, cmap='jet')
plt.scatter(coords[:, 2], coords[:, 1], s=100, marker='o', facecolors='none', edgecolor='w')

### 3D fitting

In [6]:
x, y, z = np.mgrid[
    -1.:1.:1./kernel_size*1j, 
    -1.:1.:1./kernel_size*1j, 
    -1.:1.:1./kernel_size*1j
]
# (N, 3) array of (x, y, z) pairs
xyz = np.column_stack([x.flat, y.flat, z.flat])

mu = np.array([0., 0., 0.])
sigma = np.array([.025, .025, .025])

In [ ]:
def gaussian_3d(mu, sigma):
    covariance = np.diag(sigma**2)
    z = multivariate_normal.pdf(xyz, mean=mu, cov=covariance)
    # reshape back to cubical grid
    return z.reshape(x)
x, y = np.mgrid[-1.0:1.0:30j, -1.0:1.0:30j]
# Need an (N, 2) array of (x, y) pairs.
xy = np.column_stack([x.flat, y.flat])

mu = np.array([0.0, 0.0])

sigma = np.array([.025, .025])
covariance = np.diag(sigma**2)

z = multivariate_normal.pdf(xy, mean=mu, cov=covariance)

# Reshape back to a (30, 30) grid.
z = z.reshape(x.shape)